In [9]:
# Lab | Making predictions with logistic regression

# In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

# In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

# ### Instructions

# 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. 
#   It should include some film features and some rental features. Use the data from 2005.
# 2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.
# 3. Read the data into a Pandas dataframe.
# 4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
# 5. Create a logistic regression model to predict this variable from the cleaned data.
# 6. Evaluate the results.

In [10]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import numpy as np
password = getpass.getpass()

In [11]:
# get the data
rental = pd.read_sql('rental', 'mysql+pymysql://root:' + password + '@localhost/sakila')
film = pd.read_sql('film', 'mysql+pymysql://root:' + password + '@localhost/sakila')


C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\io\sql.py:582: SAWarning: Did not recognize type 'geometry' of column 'location'
  pandas_sql.meta.reflect(bind=pandas_sql.connectable, only=[sql])


In [12]:
#1. movies
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT
	f.title
    ,f.rental_duration
    ,f.rental_rate
    ,f.length
    ,f.rating
    ,act.act_no
    ,inv.inv_no
    ,c.name AS category
    
FROM film f
	left join inventory i ON i.film_id = f.film_id
    left join rental r ON r.inventory_id = i.inventory_id
    join film_actor fa ON fa.film_id = f.film_id
    Join actor a ON a.actor_id = fa.actor_id
    join film_category fc ON fc.film_id = f.film_id
    join category c ON c.category_id = fc.category_id
    left join (SELECT count(actor_id) as act_no, film_id FROM film_actor group by film_id) act ON act.film_id = f.film_id
    left join (SELECT count(inventory_id) as inv_no, film_id FROM inventory group by film_id) inv ON inv.film_id = f.film_id
    
    WHERE year(rental_date) = 2005
    GROUP BY i.film_id;'''

fr = pd.read_sql_query(query, engine)

In [13]:
# 2. rentals
query = '''SELECT 
    title
    ,month(rental_date) AS month
FROM film f
    left join inventory i ON i.film_id = f.film_id
    left join rental r ON r.inventory_id = i.inventory_id

    WHERE year(r.rental_date) = 2005;'''
    
rental = pd.read_sql_query(query, engine)

In [14]:
# get unique movies for month may (may = 1, else = 0)
rental['month'] = np.where(rental['month']==5,1,0)
rental = rental.groupby(['title'])['month'].max()
rental = pd.DataFrame(rental).reset_index()

In [15]:
# join both tables
fr = fr.join(rental.set_index('title'), on='title')
fr

,title,rental_duration,rental_rate,length,rating,act_no,inv_no,category,month
0,ACADEMY DINOSAUR,6,0.99,86,PG,10,8,Documentary,1
1,ACE GOLDFINGER,3,4.99,48,G,4,3,Horror,0
2,ADAPTATION HOLES,7,2.99,50,NC-17,5,4,Documentary,1
3,AFFAIR PREJUDICE,5,2.99,117,G,5,7,Horror,1
4,AFRICAN EGG,6,2.99,130,G,5,3,Family,1
...,...,...,...,...,...,...,...,...,...
950,YOUNG LANGUAGE,6,0.99,183,G,5,2,Documentary,0
951,YOUTH KICK,4,0.99,179,NC-17,5,2,Music,0
952,ZHIVAGO CORE,6,0.99,105,NC-17,6,2,Horror,1
953,ZOOLANDER FICTION,5,2.99,101,R,5,5,Children,1


In [16]:
# nulls?
fr = fr.dropna()
fr.isna().sum()

title              0
rental_duration    0
rental_rate        0
length             0
rating             0
act_no             0
inv_no             0
category           0
month              0
dtype: int64

In [17]:
#X-y-SPLIT

y = fr['month']
X = fr.drop('month', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [19]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

,rental_duration,rental_rate,length,act_no,inv_no
0,0.50,1.0,0.805755,0.142857,0.000000
1,1.00,1.0,0.546763,0.071429,0.166667
2,0.25,0.5,0.244604,0.500000,0.666667
3,0.75,1.0,0.640288,0.428571,0.666667
4,0.50,0.5,0.028777,0.214286,0.500000


In [20]:
X_train

,title,rental_duration,rental_rate,length,rating,act_no,inv_no,category
292,FEVER EMPIRE,5,4.99,158,R,3,2,Games
892,VANILLA DAY,7,4.99,122,NC-17,2,3,Games
736,SEA VIRGIN,4,2.99,80,PG,8,6,Drama
109,CANDLES GRAPES,6,4.99,135,NC-17,7,6,Games
77,BLUES INSTINCT,5,2.99,50,G,4,5,Family
...,...,...,...,...,...,...,...,...
106,CAMELOT VACATION,3,0.99,61,NC-17,2,7,Sci-Fi
270,ESCAPE METROPOLIS,7,2.99,167,R,4,6,Travel
860,TRACY CIDER,3,0.99,142,G,3,7,Animation
435,INSECTS STONE,3,0.99,123,NC-17,4,6,Animation


In [21]:
from sklearn.preprocessing import OneHotEncoder
X_train_cat = X_train.select_dtypes(include=['object'])
X_train_cat = X_train_cat.drop(['title'], axis=1)

encoder = OneHotEncoder().fit(X_train_cat)
encoded = encoder.transform(X_train_cat).toarray()
cols=[colname for row in encoder.categories_ for colname in row]
onehot_encoded = pd.DataFrame(encoded,columns=cols)
cols_to_drop=[row[0] for row in encoder.categories_]
X_train_cat = onehot_encoded.drop(cols_to_drop,axis=1)

In [22]:
onehot_encoded.shape

(764, 21)

In [23]:
X_train_transformed = np.concatenate([X_train_norm, onehot_encoded], axis=1)
X_train_transformed

array([[0.5       , 1.        , 0.8057554 , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 0.54676259, ..., 0.        , 0.        ,
        0.        ],
       [0.25      , 0.5       , 0.24460432, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.69064748, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.55395683, ..., 0.        , 0.        ,
        0.        ],
       [0.25      , 0.        , 0.05035971, ..., 0.        , 0.        ,
        0.        ]])

In [24]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [36]:
# for numericals
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
# we use the transformer that was trained on the training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)
X_test_norm.head()

,0,1,2,3,4
0,0.0,1.0,0.093525,0.214286,0.333333
1,0.0,0.5,0.906475,0.357143,0.500000
2,0.0,0.0,0.000000,0.285714,1.000000
3,0.5,0.5,0.417266,0.500000,0.333333
4,1.0,0.5,0.884892,0.214286,0.166667


In [26]:
# for categoricals
X_test_cat = X_test.select_dtypes(include=['object'])
X_test_cat = X_test_cat.drop(['title'], axis=1)

encoder = OneHotEncoder().fit(X_test_cat)
encoded = encoder.transform(X_test_cat).toarray()
cols = encoder.get_feature_names(input_features=X_test_cat.columns)
onehot_encoded2 = pd.DataFrame(encoded, columns=cols)
onehot_encoded2.head()

C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,category_Action,category_Animation,category_Children,category_Classics,category_Comedy,...,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
onehot_encoded2.shape

(191, 21)

In [28]:
X_test_transformed = np.concatenate([X_test_norm, onehot_encoded2], axis=1)

In [29]:
#predictions on the test set:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.7277486910994765

In [30]:
pd.Series(predictions).value_counts()

1    172
0     19
dtype: int64

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  8,  41],
       [ 11, 131]], dtype=int64)

In [32]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf.fit(X_train_transformed, y_train)
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.6544502617801047

In [33]:
confusion_matrix(y_test, predictions_clf)

array([[  8,  41],
       [ 25, 117]], dtype=int64)

In [34]:
pd.Series(y_test).value_counts()

1    142
0     49
Name: month, dtype: int64

In [35]:
pd.Series(predictions_clf).value_counts()

1    158
0     33
dtype: int64